In [1]:
import pandas as pd
# set to display all columns
pd.set_option('display.max_columns', None)
# set to print on one line
pd.set_option('display.width', 1000)

In [2]:
dataset="MPDocVQA"

# QUR - LLM

In [3]:
weights = {'<15': 1.0, '15-25': 0.16, '>25': 0.336}

In [4]:
folders=["results_w1","results_w1_ocr"]
df_list = []
for folder in folders:
    if folder == "results_w1":
        order_colums=['Phi', 'Qwen', 'Molmo', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']
    else:
        order_colums=['Phi', 'Qwen', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']
        
    df_QUR= pd.read_csv(f'{folder}/results/QUR.csv', index_col=0)
    df_QUR=df_QUR[order_colums]
    df_QUR=df_QUR.loc[['QUR_C1','QUR_C2','QUR_C3']]
    
    df_DED_1 = pd.read_csv(f'{folder}/results/QUR_DED_complexity_1.csv', index_col=0)
    df_DED_2 = pd.read_csv(f'{folder}/results/QUR_DED_complexity_2.csv', index_col=0)
    df_DED_3 = pd.read_csv(f'{folder}/results/QUR_DED_complexity_2.csv', index_col=0)

    # create df_DED_total as empty dataframe. then for each index in df_DED_1, add the corresponding values from df_DED_1,df_DED_2 and df_DED_3 to df_DED_total. For each index, the new index is the index of df_DED_1 + 'C1', 'C2' or 'C3'
    df_DED_total = pd.DataFrame(columns=df_DED_1.columns)
    for i in df_DED_1.index:
        df_DED_total.loc[i+'_C1'] = df_DED_1.loc[i]*weights[i]
        df_DED_total.loc[i+'_C2'] = df_DED_2.loc[i]*weights[i]
        df_DED_total.loc[i+'_C3'] = df_DED_3.loc[i]*weights[i]
    # sort df_DED_total based on this list: [Qwen, InternVL, Phi, Molmo, Ovis, DocOwl
    df_DED_total=df_DED_total[order_colums]

    df_PL_1 = pd.read_csv(f'{folder}/results/QUR_PL_complexity_1.csv', index_col=0)
    df_PL_2 = pd.read_csv(f'{folder}/results/QUR_PL_complexity_2.csv', index_col=0)
    df_PL_3 = pd.read_csv(f'{folder}/results/QUR_PL_complexity_3.csv', index_col=0)
    df_PL_1 = df_PL_1.sort_index(ascending=True)
    df_PL_2 = df_PL_2.sort_index(ascending=True)
    df_PL_3 = df_PL_3.sort_index(ascending=True)

    df_PL_1 = df_PL_1.drop(index=[5,6,17,20])
    df_PL_2 = df_PL_2.drop(index=[5,6,17,20])
    df_PL_3 = df_PL_3.drop(index=[5,6,17,20])
    df_PL_1['index'] = pd.cut(df_PL_1.index, bins=[0, 4, 8, 100], labels=['<4pg', '4pg-8pg', '>8pg'])
    df_PL_1 = df_PL_1.groupby('index').mean()
    df_PL_2['index'] = pd.cut(df_PL_2.index, bins=[0, 4, 8, 100], labels=['<4pg', '4pg-8pg', '>8pg'])
    df_PL_2 = df_PL_2.groupby('index').mean()
    df_PL_3['index'] = pd.cut(df_PL_3.index, bins=[0, 4, 8, 100], labels=['<4pg', '4pg-8pg', '>8pg'])
    df_PL_3 = df_PL_3.groupby('index').mean()


    # create df_DED_total as empty dataframe. then for each index in df_DED_1, add the corresponding values from df_DED_1,df_DED_2 and df_DED_3 to df_DED_total. For each index, the new index is the index of df_DED_1 + 'C1', 'C2' or 'C3'
    df_PL_total = pd.DataFrame(columns=df_PL_1.columns)
    for i in df_PL_1.index:
        df_PL_total.loc[i+'_C1'] = df_PL_1.loc[i]
        df_PL_total.loc[i+'_C2'] = df_PL_2.loc[i]
        df_PL_total.loc[i+'_C3'] = df_PL_3.loc[i]
    # sort df_DED_total based on this list: [Qwen, InternVL, Phi, Molmo, Ovis, DocOwl
    df_PL_total=df_PL_total[order_colums]
    # df_PL_total

    df_NLPE_1 = pd.read_csv(f'{folder}/results/QUR_NLPE_complexity_1.csv', index_col=0)
    df_NLPE_2 = pd.read_csv(f'{folder}/results/QUR_NLPE_complexity_2.csv', index_col=0)
    df_NLPE_3 = pd.read_csv(f'{folder}/results/QUR_NLPE_complexity_3.csv', index_col=0)
    df_NLPE_1 = df_NLPE_1[order_colums]
    df_NLPE_2 = df_NLPE_2[order_colums]
    df_NLPE_3 = df_NLPE_3[order_colums]

    df_NLPE_total = pd.DataFrame(columns=df_NLPE_1.columns)
    for i in df_NLPE_1.index:
        df_NLPE_total.loc[i+'_C1'] = df_NLPE_1.loc[i]
        df_NLPE_total.loc[i+'_C2'] = df_NLPE_2.loc[i]
        df_NLPE_total.loc[i+'_C3'] = df_NLPE_3.loc[i]
    # sort df_DED_total based on this list: [Qwen, InternVL, Phi, Molmo, Ovis, DocOwl
    df_NLPE_total=df_NLPE_total[order_colums]


    df_total = pd.concat([df_QUR, df_DED_total, df_PL_total, df_NLPE_total], axis=0)
    if folder=="results_w1_ocr":
        # create a new column 'Molmo' with all zeros
        df_total['Molmo'] = 0
        # reorder the columns
        df_total = df_total[['Phi', 'Qwen', 'Molmo', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']]
    df_list.append(df_total)



/tmp/ipykernel_215716/2102212791.py:37: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_PL_1 = df_PL_1.groupby('index').mean()
/tmp/ipykernel_215716/2102212791.py:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_PL_2 = df_PL_2.groupby('index').mean()
/tmp/ipykernel_215716/2102212791.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_PL_3 = df_PL_3.groupby('index').mean()
/tmp/ipykernel_215716/2102212791.py:37: 

In [5]:
df_list[0]

,Phi,Qwen,Molmo,InternVL,DocOwl,Ovis,Llama,Gemma27,Llava34
QUR_C1,0.112745,0.500000,0.357843,0.215686,0.019608,0.220588,0.313725,0.401961,0.308824
QUR_C2,0.090909,0.496503,0.328671,0.258741,0.013986,0.188811,0.321678,0.419580,0.440559
QUR_C3,0.033898,0.440678,0.305085,0.186441,0.000000,0.271186,0.372881,0.305085,0.322034
<15_C1,0.100000,0.494444,0.366667,0.216667,0.022222,0.227778,0.316667,0.383333,0.277778
<15_C2,0.085271,0.519380,0.341085,0.255814,0.015504,0.193798,0.317829,0.441860,0.449612
<15_C3,0.085271,0.519380,0.341085,0.255814,0.015504,0.193798,0.317829,0.441860,0.449612
15-25_C1,0.020000,0.040000,0.020000,0.020000,0.000000,0.020000,0.040000,0.120000,0.120000
15-25_C2,0.000000,0.000000,0.032000,0.000000,0.000000,0.032000,0.032000,0.000000,0.064000
15-25_C3,0.000000,0.000000,0.032000,0.000000,0.000000,0.032000,0.032000,0.000000,0.064000
>25_C1,0.084000,0.231000,0.126000,0.084000,0.000000,0.063000,0.105000,0.147000,0.147000


In [6]:
df_list[1]

,Phi,Qwen,Molmo,InternVL,DocOwl,Ovis,Llama,Gemma27,Llava34
QUR_C1,0.196078,0.598039,0,0.299020,0.029412,0.450980,0.387255,0.377451,0.289216
QUR_C2,0.153846,0.573427,0,0.349650,0.027972,0.223776,0.349650,0.300699,0.328671
QUR_C3,0.169492,0.559322,0,0.271186,0.016949,0.220339,0.322034,0.305085,0.271186
<15_C1,0.177778,0.594444,0,0.300000,0.027778,0.444444,0.394444,0.344444,0.266667
<15_C2,0.147287,0.565891,0,0.348837,0.031008,0.240310,0.348837,0.279070,0.325581
<15_C3,0.147287,0.565891,0,0.348837,0.031008,0.240310,0.348837,0.279070,0.325581
15-25_C1,0.040000,0.080000,0,0.040000,0.000000,0.040000,0.060000,0.100000,0.080000
15-25_C2,0.032000,0.064000,0,0.032000,0.000000,0.000000,0.032000,0.064000,0.032000
15-25_C3,0.032000,0.064000,0,0.032000,0.000000,0.000000,0.032000,0.064000,0.032000
>25_C1,0.126000,0.231000,0,0.105000,0.021000,0.210000,0.105000,0.210000,0.147000


In [7]:
# create df_final which has twice the columns of df_list[0] and df_list[1] and it is made by altering the columns of df_list[0] and df_list[1]
df_final = pd.DataFrame()
# add the columns of df_list[0] and df_list[1] to df_final in alternating order
for i in range(len(df_list[0].columns)):
    df_final[df_list[0].columns[i] + '_w1'] = df_list[0].iloc[:, i]
    df_final[df_list[1].columns[i] + '_w1_ocr'] = df_list[1].iloc[:, i]
df_final

,Phi_w1,Phi_w1_ocr,Qwen_w1,Qwen_w1_ocr,Molmo_w1,Molmo_w1_ocr,InternVL_w1,InternVL_w1_ocr,DocOwl_w1,DocOwl_w1_ocr,Ovis_w1,Ovis_w1_ocr,Llama_w1,Llama_w1_ocr,Gemma27_w1,Gemma27_w1_ocr,Llava34_w1,Llava34_w1_ocr
QUR_C1,0.112745,0.196078,0.500000,0.598039,0.357843,0,0.215686,0.299020,0.019608,0.029412,0.220588,0.450980,0.313725,0.387255,0.401961,0.377451,0.308824,0.289216
QUR_C2,0.090909,0.153846,0.496503,0.573427,0.328671,0,0.258741,0.349650,0.013986,0.027972,0.188811,0.223776,0.321678,0.349650,0.419580,0.300699,0.440559,0.328671
QUR_C3,0.033898,0.169492,0.440678,0.559322,0.305085,0,0.186441,0.271186,0.000000,0.016949,0.271186,0.220339,0.372881,0.322034,0.305085,0.305085,0.322034,0.271186
<15_C1,0.100000,0.177778,0.494444,0.594444,0.366667,0,0.216667,0.300000,0.022222,0.027778,0.227778,0.444444,0.316667,0.394444,0.383333,0.344444,0.277778,0.266667
<15_C2,0.085271,0.147287,0.519380,0.565891,0.341085,0,0.255814,0.348837,0.015504,0.031008,0.193798,0.240310,0.317829,0.348837,0.441860,0.279070,0.449612,0.325581
<15_C3,0.085271,0.147287,0.519380,0.565891,0.341085,0,0.255814,0.348837,0.015504,0.031008,0.193798,0.240310,0.317829,0.348837,0.441860,0.279070,0.449612,0.325581
15-25_C1,0.020000,0.040000,0.040000,0.080000,0.020000,0,0.020000,0.040000,0.000000,0.000000,0.020000,0.040000,0.040000,0.060000,0.120000,0.100000,0.120000,0.080000
15-25_C2,0.000000,0.032000,0.000000,0.064000,0.032000,0,0.000000,0.032000,0.000000,0.000000,0.032000,0.000000,0.032000,0.032000,0.000000,0.064000,0.064000,0.032000
15-25_C3,0.000000,0.032000,0.000000,0.064000,0.032000,0,0.000000,0.032000,0.000000,0.000000,0.032000,0.000000,0.032000,0.032000,0.000000,0.064000,0.064000,0.032000
>25_C1,0.084000,0.126000,0.231000,0.231000,0.126000,0,0.084000,0.105000,0.000000,0.021000,0.063000,0.210000,0.105000,0.105000,0.147000,0.210000,0.147000,0.147000


In [8]:
df_final.to_latex(f'table/{dataset}_QUR_ABLATION.tex',index=True, float_format="%.3f", escape=False)

# UR - LLM

In [9]:
folders=["results_w1","results_w1_ocr"]
df_list = []
for folder in folders:
    if folder == "results_w1":
        order_colums=['Phi', 'Qwen', 'Molmo', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']
    else:
        order_colums=['Phi', 'Qwen', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']
        
    df_QUR= pd.read_csv(f'{folder}/results/UR.csv', index_col=0)
    df_QUR=df_QUR[order_colums]
    df_QUR=df_QUR.loc[['UR_C1','UR_C2','UR_C3']]
    
    df_DED_1 = pd.read_csv(f'{folder}/results/UR_PAGE_DED_complexity_1.csv', index_col=0)
    df_DED_2 = pd.read_csv(f'{folder}/results/UR_PAGE_DED_complexity_2.csv', index_col=0)
    df_DED_3 = pd.read_csv(f'{folder}/results/UR_PAGE_DED_complexity_2.csv', index_col=0)

    # create df_DED_total as empty dataframe. then for each index in df_DED_1, add the corresponding values from df_DED_1,df_DED_2 and df_DED_3 to df_DED_total. For each index, the new index is the index of df_DED_1 + 'C1', 'C2' or 'C3'
    df_DED_total = pd.DataFrame(columns=df_DED_1.columns)
    for i in df_DED_1.index:
        df_DED_total.loc[i+'_C1'] = df_DED_1.loc[i]
        df_DED_total.loc[i+'_C2'] = df_DED_2.loc[i]
        df_DED_total.loc[i+'_C3'] = df_DED_3.loc[i]
    # sort df_DED_total based on this list: [Qwen, InternVL, Phi, Molmo, Ovis, DocOwl
    df_DED_total=df_DED_total[order_colums]

    df_IP= pd.read_csv(f'{folder}/results/UR_PAGE_inpage.csv', index_col=0)
    df_IP=df_IP[order_colums]
    df_IP=df_IP.loc[['UR_inpage_C1','UR_inpage_C2','UR_inpage_C3']]
    df_OP= pd.read_csv(f'{folder}/results/UR_PAGE_outpage.csv', index_col=0)
    df_OP=df_OP[order_colums]
    df_OP=df_OP.loc[['UR_outpage_C1','UR_outpage_C2','UR_outpage_C3']]

    df_NLPE_1 = pd.read_csv(f'{folder}/results/UR_NLPE_complexity_1.csv', index_col=0)
    df_NLPE_2 = pd.read_csv(f'{folder}/results/UR_NLPE_complexity_2.csv', index_col=0)
    df_NLPE_3 = pd.read_csv(f'{folder}/results/UR_NLPE_complexity_3.csv', index_col=0)
    df_NLPE_1 = df_NLPE_1[order_colums]
    df_NLPE_2 = df_NLPE_2[order_colums]
    df_NLPE_3 = df_NLPE_3[order_colums]

    df_NLPE_total = pd.DataFrame(columns=df_NLPE_1.columns)
    for i in df_NLPE_1.index:
        df_NLPE_total.loc[i+'_C1'] = df_NLPE_1.loc[i]
        df_NLPE_total.loc[i+'_C2'] = df_NLPE_2.loc[i]
        df_NLPE_total.loc[i+'_C3'] = df_NLPE_3.loc[i]
    # sort df_DED_total based on this list: [Qwen, InternVL, Phi, Molmo, Ovis, DocOwl
    df_NLPE_total=df_NLPE_total[order_colums]


    df_total = pd.concat([df_QUR, df_DED_total, df_IP, df_OP, df_NLPE_total], axis=0)
    if folder=="results_w1_ocr":
        # create a new column 'Molmo' with all zeros
        df_total['Molmo'] = 0
        # reorder the columns
        df_total = df_total[['Phi', 'Qwen', 'Molmo', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']]
    df_list.append(df_total)



In [10]:
# create df_final which has twice the columns of df_list[0] and df_list[1] and it is made by altering the columns of df_list[0] and df_list[1]
df_final = pd.DataFrame()
# add the columns of df_list[0] and df_list[1] to df_final in alternating order
for i in range(len(df_list[0].columns)):
    df_final[df_list[0].columns[i] + '_w1'] = df_list[0].iloc[:, i]
    df_final[df_list[1].columns[i] + '_w1_ocr'] = df_list[1].iloc[:, i]
df_final

,Phi_w1,Phi_w1_ocr,Qwen_w1,Qwen_w1_ocr,Molmo_w1,Molmo_w1_ocr,InternVL_w1,InternVL_w1_ocr,DocOwl_w1,DocOwl_w1_ocr,Ovis_w1,Ovis_w1_ocr,Llama_w1,Llama_w1_ocr,Gemma27_w1,Gemma27_w1_ocr,Llava34_w1,Llava34_w1_ocr
UR_C1,0.525754,0.795854,0.901382,0.924623,0.830402,0,0.805905,0.857412,0.288945,0.339196,0.814698,0.681533,0.844849,0.857412,0.852387,0.882538,0.707286,0.849246
UR_C2,0.540767,0.687050,0.850120,0.887290,0.699041,0,0.754197,0.804556,0.231415,0.294964,0.739808,0.758993,0.724221,0.743405,0.823741,0.806954,0.729017,0.762590
UR_C3,0.575130,0.751295,0.865285,0.888601,0.748705,0,0.813472,0.823834,0.189119,0.220207,0.808290,0.813472,0.748705,0.795337,0.808290,0.823834,0.663212,0.797927
0_C1,0.579387,0.774373,0.888579,0.917827,0.814763,0,0.818942,0.857939,0.341226,0.431755,0.857939,0.614206,0.828691,0.855153,0.874652,0.869081,0.732591,0.837047
0_C2,0.582609,0.723913,0.882609,0.921739,0.708696,0,0.834783,0.850000,0.247826,0.371739,0.808696,0.821739,0.728261,0.786957,0.869565,0.841304,0.732609,0.804348
0_C3,0.582609,0.723913,0.882609,0.921739,0.708696,0,0.834783,0.850000,0.247826,0.371739,0.808696,0.821739,0.728261,0.786957,0.869565,0.841304,0.732609,0.804348
1_C1,0.525758,0.854545,0.931818,0.943939,0.850000,0,0.815152,0.872727,0.253030,0.283333,0.825758,0.703030,0.875758,0.868182,0.846970,0.915152,0.698485,0.878788
1_C2,0.486111,0.604167,0.795139,0.829861,0.649306,0,0.607639,0.718750,0.177083,0.194444,0.625000,0.652778,0.687500,0.652778,0.746528,0.743056,0.715278,0.680556
1_C3,0.486111,0.604167,0.795139,0.829861,0.649306,0,0.607639,0.718750,0.177083,0.194444,0.625000,0.652778,0.687500,0.652778,0.746528,0.743056,0.715278,0.680556
>1_C1,0.345794,0.686916,0.850467,0.887850,0.822430,0,0.733645,0.808411,0.224299,0.200935,0.635514,0.841121,0.803738,0.831776,0.794393,0.827103,0.649533,0.799065


In [11]:
df_final.to_latex(f'table/{dataset}_UR_ABLATION.tex',index=True, float_format="%.3f", escape=False)

# INPAGE

In [14]:
folders=["results_w1","results_w1_ocr"]
df_list = []
for folder in folders:
    if folder == "results_w1":
        order_colums=['Phi', 'Qwen', 'Molmo', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']
    else:
        order_colums=['Phi', 'Qwen', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']
        
    
    df_QP_1 = pd.read_csv(f'{folder}/results/UR_PAGE_QP_complexity_1.csv', index_col=0)
    df_QP_2 = pd.read_csv(f'{folder}/results/UR_PAGE_QP_complexity_2.csv', index_col=0)
    df_QP_3 = pd.read_csv(f'{folder}/results/UR_PAGE_QP_complexity_2.csv', index_col=0)

    # create df_DED_total as empty dataframe. then for each index in df_DED_1, add the corresponding values from df_DED_1,df_DED_2 and df_DED_3 to df_DED_total. For each index, the new index is the index of df_DED_1 + 'C1', 'C2' or 'C3'
    df_QP_total = pd.DataFrame(columns=df_QP_1.columns)
    for i in df_QP_1.index:
        df_QP_total.loc[i+'_C1'] = df_QP_1.loc[i]
        df_QP_total.loc[i+'_C2'] = df_QP_2.loc[i]
        df_QP_total.loc[i+'_C3'] = df_QP_3.loc[i]
    # sort df_DED_total based on this list: [Qwen, InternVL, Phi, Molmo, Ovis, DocOwl
    df_QP_total=df_QP_total[order_colums]

    df_DED_1 = pd.read_csv(f'{folder}/results/UR_PAGE_DE_complexity_1.csv', index_col=0)
    df_DED_1 = df_DED_1[order_colums]
    df_DED_1 = df_DED_1.loc[['title', 'plain text', 'figure', 'table', 'abandon']]
    df_DED_2 = pd.read_csv(f'{folder}/results/UR_PAGE_DE_complexity_2.csv', index_col=0)
    df_DED_2 = df_DED_2[order_colums]
    df_DED_2 = df_DED_2.loc[['title', 'plain text', 'figure', 'table', 'abandon']]
    df_DED_3 = pd.read_csv(f'{folder}/results/UR_PAGE_DE_complexity_3.csv', index_col=0)
    df_DED_3 = df_DED_3[order_colums]
    df_DED_3 = df_DED_3.loc[['title', 'plain text', 'figure', 'table', 'abandon']]

    df_DED_total = pd.DataFrame(columns=df_DED_1.columns)
    for i in df_DED_1.index:
        df_DED_total.loc[i+'_C1'] = df_DED_1.loc[i]
        df_DED_total.loc[i+'_C2'] = df_DED_2.loc[i]
        df_DED_total.loc[i+'_C3'] = df_DED_3.loc[i]
    # sort df_DED_total based on this list: [Qwen, InternVL, Phi, Molmo, Ovis, DocOwl
    df_DED_total=df_DED_total[order_colums]

    df_total = pd.concat([df_DED_total, df_QP_total], axis=0)
    if folder=="results_w1_ocr":
        # create a new column 'Molmo' with all zeros
        df_total['Molmo'] = 0
        # reorder the columns
        df_total = df_total[['Phi', 'Qwen', 'Molmo', 'InternVL', 'DocOwl', 'Ovis', 'Llama', 'Gemma27', 'Llava34']]
    df_list.append(df_total)



In [15]:
# create df_final which has twice the columns of df_list[0] and df_list[1] and it is made by altering the columns of df_list[0] and df_list[1]
df_final = pd.DataFrame()
# add the columns of df_list[0] and df_list[1] to df_final in alternating order
for i in range(len(df_list[0].columns)):
    df_final[df_list[0].columns[i] + '_w1'] = df_list[0].iloc[:, i]
    df_final[df_list[1].columns[i] + '_w1_ocr'] = df_list[1].iloc[:, i]
df_final

,Phi_w1,Phi_w1_ocr,Qwen_w1,Qwen_w1_ocr,Molmo_w1,Molmo_w1_ocr,InternVL_w1,InternVL_w1_ocr,DocOwl_w1,DocOwl_w1_ocr,Ovis_w1,Ovis_w1_ocr,Llama_w1,Llama_w1_ocr,Gemma27_w1,Gemma27_w1_ocr,Llava34_w1,Llava34_w1_ocr
title_C1,0.250000,0.250000,0.250000,0.750000,0.250000,0,0.250000,0.250000,0.000000,0.000000,0.250000,0.250000,0.250000,0.500000,0.750000,0.250000,0.750000,0.250000
title_C2,0.000000,0.066667,0.266667,0.600000,0.333333,0,0.333333,0.333333,0.200000,0.066667,0.333333,0.266667,0.133333,0.533333,0.266667,0.666667,0.533333,0.666667
title_C3,0.411765,0.705882,0.823529,0.882353,0.588235,0,0.588235,0.647059,0.058824,0.000000,0.470588,0.529412,0.647059,0.647059,0.823529,0.882353,0.764706,0.764706
plain text_C1,0.480447,0.653631,0.843575,0.893855,0.675978,0,0.614525,0.720670,0.223464,0.279330,0.782123,0.558659,0.720670,0.787709,0.843575,0.782123,0.726257,0.715084
plain text_C2,0.449541,0.591743,0.834862,0.853211,0.628440,0,0.665138,0.724771,0.192661,0.302752,0.642202,0.692661,0.646789,0.697248,0.788991,0.711009,0.775229,0.688073
plain text_C3,0.419643,0.553571,0.767857,0.803571,0.535714,0,0.705357,0.705357,0.160714,0.178571,0.723214,0.732143,0.526786,0.660714,0.750000,0.696429,0.607143,0.625000
figure_C1,0.322581,0.483871,0.838710,0.806452,0.612903,0,0.290323,0.580645,0.032258,0.096774,0.387097,0.354839,0.677419,0.677419,0.774194,0.935484,0.709677,0.709677
figure_C2,0.488372,0.534884,0.790698,0.883721,0.581395,0,0.488372,0.720930,0.116279,0.116279,0.325581,0.534884,0.674419,0.651163,0.511628,0.674419,0.720930,0.697674
figure_C3,0.454545,0.272727,0.727273,0.818182,0.454545,0,0.363636,0.454545,0.272727,0.090909,0.363636,0.363636,0.454545,0.545455,0.636364,0.545455,0.545455,0.363636
table_C1,0.310345,0.862069,1.000000,1.000000,0.862069,0,0.689655,0.724138,0.103448,0.068966,0.655172,0.758621,0.827586,0.896552,0.827586,0.965517,0.758621,0.896552


In [16]:
df_final.to_latex(f'table/{dataset}_UR_INPAGE_ABLATION.tex',index=True, float_format="%.3f", escape=False)

# OLD

In [ ]:
metrics ={
    "QUR": ["QUR""],
    "QUR_DED": ["<15", "15-20", "UR_C3"],	
    "OPAR": ["Percentage_C1", "Percentage_C2", "Percentage_C3"],
    "ANSL": ["ANSL_C1", "ANSL_C2", "ANSL_C3"],
    "CEPAR": ["Percentage_C1", "Percentage_C2", "Percentage_C3"],
    "QEWR": ["Percentage_C1", "Percentage_C2", "Percentage_C3"],
}

In [ ]:
final_results = pd.DataFrame()
for key, value in metrics.items():
    metric_results = pd.DataFrame()
    print(f"Metric: {key}")
    for model in ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']:
        data_model = []
        print(f"\t\tModel: {model}")
        for folder in ["results_w1", "results_w1_ocr", "results_w1_unable", "results_w1_ocr_unable"]:
            print(f"\t\t\tFolder: {folder}")
            df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{key}.csv', index_col=0)
            if "ocr" in folder:
                df_QUR_W2['Molmo'] = [0 for el in df_QUR_W2['Qwen'].values]
            df_QUR_W2 = df_QUR_W2[['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']]
            # select the model
            df_QUR_W2 = df_QUR_W2[[model]]
            data=[]
            for k in value:
                data.append(df_QUR_W2.loc[df_QUR_W2.index == k].values[0][0])
            data_model.append(data)

        # print(f"Model: {model}")
        # make the dataframe
        data_model = pd.DataFrame(data_model, columns=["C1", "C2", "C3"])
        data_model.index = ["Explicit", "OCR + Excplicit", "None", "OCR"]
        # pivot the dataframe
        data_model = data_model.T
        # print(data_model)
        metric_results = pd.concat([metric_results, data_model], axis=1)
    # set external index with the metric name
    metric_results.index = pd.MultiIndex.from_product([[key], metric_results.index])
    final_results = pd.concat([final_results, metric_results], axis=0)

# print(final_results)
categories = ["Explicit", "OCR + Explicit", "None", "OCR"]
models = ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']
column_tuples = [(model, category) for model in models for category in categories]
final_results.columns = pd.MultiIndex.from_tuples(column_tuples, names=['', ''])

final_results = final_results.round(3)

# print(final_results)
print(final_results.to_latex())
# save the results in latex
final_results.to_latex(f"tables/{dataset}_results.tex")

In [ ]:
final_results = pd.DataFrame()
layouts = ["plain text", "figure", "table", "title", "abandon"]
final_results = pd.DataFrame()
metric="CEPAR_PLT"
for lay in layouts:
    layout_results = pd.DataFrame()
    print(f"Layout: {lay}")
    for model in ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']:
        print(f"\t\tModel: {model}")
        data=[]
        for folder in ["results_w1", "results_w1_ocr", "results_w1_unable", "results_w1_ocr_unable"]:
            print(f"\t\t\tFolder: {folder}")
            type=["","C1","C2","C3"]
            tmp=[]
            for t in type:
                if t == "":
                    df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{metric}.csv', index_col=0)
                else:
                    df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{metric}_{t}.csv', index_col=0)
                # df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{key}.csv', index_col=0)
                if "ocr" in folder:
                    df_QUR_W2['Molmo'] = [0 for el in df_QUR_W2['Qwen'].values]
                df_QUR_W2 = df_QUR_W2[['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']]
                # select the model
                df_QUR_W2 = df_QUR_W2[[model]]
                tmp.append(df_QUR_W2.loc[df_QUR_W2.index == lay].values[0][0])
            data.append(tmp)

        # print(f"Model: {model}")
        # make the dataframe
        data_model = pd.DataFrame(data, columns=["TOT","C1", "C2", "C3"])
        print(data_model)
        data_model.index = ["Explic", "OCR\nExcplic", "None", "OCR"]
        # pivot the dataframe
        data_model = data_model.T
        # print(data_model)
        layout_results = pd.concat([layout_results, data_model], axis=1)
    # set external index with the metric name
    layout_results.index = pd.MultiIndex.from_product([[lay], layout_results.index])
    final_results = pd.concat([final_results, layout_results], axis=0)

# print(final_results)
categories = ["Explic", "OCR\nExplic", "None", "OCR"]
models = ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']
column_tuples = [(model, category) for model in models for category in categories]
final_results.columns = pd.MultiIndex.from_tuples(column_tuples, names=['', ''])

final_results = final_results.round(3)

# print(final_results)
print(final_results.to_latex())
# save the results in latex
final_results.to_latex(f"tables/{dataset}_results_CEPAR_layout.tex")

In [ ]:
final_results = pd.DataFrame()
layouts = ["Match Entity Rate", "Match Layout Rate", "Hallucination Rate"]
final_results = pd.DataFrame()
metric="AEMR_ALMR_HR"
for lay in layouts:
    layout_results = pd.DataFrame()
    print(f"Layout: {lay}")
    for model in ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']:
        print(f"\t\tModel: {model}")
        data=[]
        for folder in ["results_w1", "results_w1_ocr", "results_w1_unable", "results_w1_ocr_unable"]:
            print(f"\t\t\tFolder: {folder}")
            type=["","C1","C2","C3"]
            tmp=[]
            for t in type:
                if t == "":
                    df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{metric}.csv', index_col=0)
                else:
                    df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{metric}_{t}.csv', index_col=0)
                # df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{key}.csv', index_col=0)
                if "ocr" in folder:
                    df_QUR_W2['Molmo'] = [0 for el in df_QUR_W2['Qwen'].values]
                df_QUR_W2 = df_QUR_W2[['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']]
                # select the model
                df_QUR_W2 = df_QUR_W2[[model]]
                tmp.append(df_QUR_W2.loc[df_QUR_W2.index == lay].values[0][0])
            data.append(tmp)

        # print(f"Model: {model}")
        # make the dataframe
        data_model = pd.DataFrame(data, columns=["TOT","C1", "C2", "C3"])
        print(data_model)
        data_model.index = ["Explic", "OCR\nExcplic", "None", "OCR"]
        # pivot the dataframe
        data_model = data_model.T
        # print(data_model)
        layout_results = pd.concat([layout_results, data_model], axis=1)
    # set external index with the metric name
    layout_results.index = pd.MultiIndex.from_product([[lay], layout_results.index])
    final_results = pd.concat([final_results, layout_results], axis=0)

# print(final_results)
categories = ["Explic", "OCR\nExplic", "None", "OCR"]
models = ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']
column_tuples = [(model, category) for model in models for category in categories]
final_results.columns = pd.MultiIndex.from_tuples(column_tuples, names=['', ''])

final_results = final_results.round(3)

# print(final_results)
print(final_results.to_latex())
# save the results in latex
final_results.to_latex(f"tables/{dataset}_results_{metric}.tex")

In [ ]:
final_results = pd.DataFrame()
layouts = ["plain text", "figure", "table", "title", "abandon"]
final_results = pd.DataFrame()
metric="CEPAR_PLT"
for lay in layouts:
    layout_results = pd.DataFrame()
    print(f"Layout: {lay}")
    for model in ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']:
        print(f"\t\tModel: {model}")
        data=[]
        for folder in ["results_w1", "results_w1_ocr", "results_w1_unable", "results_w1_ocr_unable"]:
            print(f"\t\t\tFolder: {folder}")
            type=["","C1","C2","C3"]
            tmp=[]
            for t in type:
                if t == "":
                    df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{metric}.csv', index_col=0)
                else:
                    df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{metric}_{t}.csv', index_col=0)
                # df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{key}.csv', index_col=0)
                if "ocr" in folder:
                    df_QUR_W2['Molmo'] = [0 for el in df_QUR_W2['Qwen'].values]
                df_QUR_W2 = df_QUR_W2[['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']]
                # select the model
                df_QUR_W2 = df_QUR_W2[[model]]
                tmp.append(df_QUR_W2.loc[df_QUR_W2.index == lay].values[0][0])
            data.append(tmp)

        # print(f"Model: {model}")
        # make the dataframe
        data_model = pd.DataFrame(data, columns=["TOT","C1", "C2", "C3"])
        print(data_model)
        data_model.index = ["Explic", "OCR\nExcplic", "None", "OCR"]
        # pivot the dataframe
        data_model = data_model.T
        # print(data_model)
        layout_results = pd.concat([layout_results, data_model], axis=1)
    # set external index with the metric name
    layout_results.index = pd.MultiIndex.from_product([[lay], layout_results.index])
    final_results = pd.concat([final_results, layout_results], axis=0)

# print(final_results)
categories = ["Explic", "OCR\nExplic", "None", "OCR"]
models = ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']
column_tuples = [(model, category) for model in models for category in categories]
final_results.columns = pd.MultiIndex.from_tuples(column_tuples, names=['', ''])

final_results = final_results.round(3)

# print(final_results)
print(final_results.to_latex())
# save the results in latex
final_results.to_latex(f"tables/{dataset}_results_CEPAR_layout.tex")

In [3]:
final_results = pd.DataFrame()
layouts = ["title", "plain text", "abandon", "figure", "table"]
final_results = pd.DataFrame()
metric="DEWP"
for lay in layouts:
    layout_results = pd.DataFrame()
    # print(f"Layout: {lay}")
    for model in ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']:
        print(f"\t\tModel: {model}")
        data=[]
        for folder in ["results_w1", "results_w1_ocr", "results_w1_unable", "results_w1_ocr_unable"]:
            print(f"\t\t\tFolder: {folder}")
            type=["","C1","C2","C3"]
            tmp=[]
            for t in type:
                if t == "":
                    df_QUR_W2 = pd.read_csv(f'{folder}/results/{metric}.csv', index_col=0)
                else:
                    df_QUR_W2 = pd.read_csv(f'{folder}/results/{metric}_{t}.csv', index_col=0)
                # df_QUR_W2 = pd.read_csv(f'{folder}/results/{dataset}_{key}.csv', index_col=0)
                if "ocr" in folder:
                    df_QUR_W2['Molmo'] = [0 for el in df_QUR_W2['Qwen'].values]
                df_QUR_W2 = df_QUR_W2[['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']]
                # select the model
                df_QUR_W2 = df_QUR_W2[[model]]
                tmp.append(df_QUR_W2.loc[df_QUR_W2.index == lay].values[0][0])
            data.append(tmp)

        # print(f"Model: {model}")
        # make the dataframe
        data_model = pd.DataFrame(data, columns=["TOT","C1", "C2", "C3"])
        print(data_model)
        data_model.index = ["Explic", "OCR\nExcplic", "None", "OCR"]
        # pivot the dataframe
        data_model = data_model.T
        # print(data_model)
        layout_results = pd.concat([layout_results, data_model], axis=1)
    # set external index with the metric name
    layout_results.index = pd.MultiIndex.from_product([[lay], layout_results.index])
    final_results = pd.concat([final_results, layout_results], axis=0)

# print(final_results)
categories = ["Explic", "OCR\nExplic", "None", "OCR"]
models = ['Qwen', 'InternVL', 'Phi', 'Molmo', 'Ovis', 'DocOwl']
column_tuples = [(model, category) for model in models for category in categories]
final_results.columns = pd.MultiIndex.from_tuples(column_tuples, names=['', ''])

final_results = final_results.round(3)

# print(final_results)
print(final_results.to_latex())
# save the results in latex
final_results.to_latex(f"tables/{dataset}_results_{metric}.tex")

		Model: Qwen
			Folder: results_w1
			Folder: results_w1_ocr
			Folder: results_w1_unable
			Folder: results_w1_ocr_unable
        TOT    C1     C2   C3
0  0.785714  1.00  0.750  0.5
1  0.857143  1.00  0.875  0.5
2  0.142857  0.25  0.125  0.0
3  0.285714  0.50  0.250  0.0
		Model: InternVL
			Folder: results_w1
			Folder: results_w1_ocr
			Folder: results_w1_unable
			Folder: results_w1_ocr_unable
        TOT    C1     C2   C3
0  0.428571  0.75  0.250  0.5
1  0.571429  1.00  0.375  0.5
2  0.142857  0.25  0.125  0.0
3  0.285714  0.75  0.125  0.0
		Model: Phi
			Folder: results_w1
			Folder: results_w1_ocr
			Folder: results_w1_unable
			Folder: results_w1_ocr_unable
        TOT    C1     C2   C3
0  0.500000  1.00  0.125  1.0
1  0.428571  1.00  0.125  0.5
2  0.000000  0.00  0.000  0.0
3  0.142857  0.25  0.000  0.5
		Model: Molmo
			Folder: results_w1
			Folder: results_w1_ocr
			Folder: results_w1_unable
			Folder: results_w1_ocr_unable
        TOT    C1     C2   C3
0  0.357143  0.75  0